In [1]:
# Importing necessary libraries
import os
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
import random

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Delete punctuation from a string
s = str.maketrans('', '', punctuation)
a = 'salam!@'
a.translate(s)

In [ ]:
# Download stopwords
stop_words = stopwords.words('english')

In [ ]:
# Read negative documents
negative_documents = []
max_len_negative = 0 
for file in os.listdir('data/neg'): # folder
    with open('data/neg/' + file) as f: # open each file
        text = f.read() 
        tokens = word_tokenize(text)
        translator = str.maketrans('', '', punctuation) # delete punctuation
        tokens = [w.translate(translator) for w in tokens] 
        tokens = [w for w in tokens if not w in stop_words]
        if len(tokens) > max_len_negative:
            max_len_negative = len(tokens) 
        negative_documents.append(' '.join(tokens)) # join all file in one
len(negative_documents)

In [ ]:
max_len_negative

In [ ]:
positive_documents = []
max_len_positive = 0
for file in os.listdir('data/pos'):
    with open('data/pos/' + file) as f:
        text = f.read()
        tokens = word_tokenize(text)
        translator = str.maketrans('', '', punctuation)
        tokens = [w.translate(translator) for w in tokens]
        tokens = [w for w in tokens if not w in stop_words]
        if len(tokens) > max_len_positive:
            max_len_positive = len(tokens) # find largest file
        positive_documents.append(' '.join(tokens))
len(positive_documents)

In [ ]:
max_len_positive

In [ ]:
max_len = max(max_len_negative, max_len_positive) 

In [ ]:
random.shuffle(negative_documents) 
random.shuffle(positive_documents)

In [ ]:
X_train = negative_documents[:800] + positive_documents[:800]

In [ ]:
len(X_train)

In [ ]:
y_train = [0 for _ in range(800)] + [1 for _ in range(800)] # add class to each text ( give 0 class to first 800 )

In [ ]:
len(y_train)

In [ ]:
X_test = negative_documents[800:] + positive_documents[800:]
len(X_test)

In [ ]:
y_test = [0 for _ in range(200)] + [1 for _ in range(200)]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Embedding, Conv1D, MaxPool1D, Dropout
from tensorflow.keras.layers import concatenate

In [ ]:
max_len

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
import pickle
with open('tokenizer.h5', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
tokenizer

In [ ]:
vocab_len = len(tokenizer.word_index) + 1 # count of unique word

In [ ]:
vocab_len

In [ ]:
encoded = tokenizer.texts_to_sequences(X_train) # encoding for embedding(show each word in n dimmision)  (replace the word with number of that word)

In [ ]:
encoded[0]

In [ ]:
padded = pad_sequences(encoded, maxlen=max_len, padding='post') # expand all text to largest text -- add 0 to end of them 

In [ ]:
padded.shape

In [ ]:
# embedding (show each word in n dimmision) if we take them to decart plan --> words that have nearest together --> nearest to gether in decart plan 
input1 = Input(shape=(max_len,)) # this is a new way to define model (at the end of each other we must give it its input ) 
embedding1 = Embedding(vocab_len, 100)(input1) # count of dimmision
conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1) # Conv1D usually for text
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPool1D(pool_size=2)(drop1)
flat1 = Flatten()(pool1)
dense1 = Dense(10, activation='relu')(flat1)
output = Dense(1, activation='sigmoid')(dense1)

In [ ]:
model = Model(inputs=[input1], outputs=output) # introduce input and output

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Display model summary
model.summary()

In [ ]:
# Tokenize and pad test data
encoded_test = tokenizer.texts_to_sequences(X_test)
padded_test = pad_sequences(encoded_test, maxlen=max_len, padding='post')

In [ ]:
# Display shape of padded test data
padded_test.shape

In [ ]:
# Import numpy
import numpy as np

In [ ]:
# Fit the model on training data
model.fit([padded], np.array(y_train), epochs=10, batch_size=20, validation_data=([padded_test], np.array(y_test)))

In [ ]:
# Save the trained model
model.save('textcnn.h5')

In [ ]:
# Plot the model architecture
plot_model(model, show_shapes=True)

In [ ]:
# Load the saved model
from tensorflow.keras.models import load_model

In [ ]:
# Load the saved tokenizer
model = load_model('textcnn.h5')

In [ ]:
# Load the saved tokenizer
with open('tokenizer.h5', 'rb') as f:
    tokenizer = pickle.load(f)

In [ ]:
# Display the word index of the tokenizer
tokenizer.word_index

In [ ]:
# Predict using the loaded model
model.predict()